In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


In [ ]:
fake = pd.read_csv(r'../input/fake-and-real-news-dataset/Fake.csv')
true = pd.read_csv(r'../input/fake-and-real-news-dataset/True.csv')

In [ ]:
pd.set_option('display.max_colwidth',300)

In [ ]:
fake.shape, true.shape

In [ ]:
fake.head()

In [ ]:
length_fake = fake['title'].str.len()
length_true = true['title'].str.len()
plt.hist(length_fake,bins=20,label='fake_title')
plt.hist(length_true,bins=20,label='true_title')
plt.legend()
plt.show()

In [ ]:
fake['label'] = 1
true['label'] = 0
data = pd.concat([fake,true],ignore_index=True)

In [ ]:
data.head()

In [ ]:
data[data['label']==1].head()

In [ ]:
data[data['label']==0].head()

In [ ]:
data.nunique()

In [ ]:
# Removing subject and date columns:
data.drop('date',axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
X = data.drop('label',axis=1)
y = data['label']

In [ ]:
### Vocabulary size
voc_size=10000

#### **Onehot Representation**

In [ ]:
messages=X.copy()
messages['title'][1]

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]

#### **Embedding Representation**

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [ ]:
embedded_docs[0]

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
len(embedded_docs),y.shape

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

#### **Model Training**

In [ ]:
### Finally Training
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=128)

#### **Performance Metrics And Accuracy**

In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
print(f'confusion_matrix: {confusion_matrix(y_test,y_pred)}')

In [ ]:
from sklearn.metrics import accuracy_score
print(f'accuracy_score: {accuracy_score(y_test,y_pred)}')